# Варианты

1. Написать правила с помощью natasha/yargy - Тут много ручной работы при составлении правил и грамматических шаблонов. При таком методе можно получить хорошие резульаты для брендов, однако сложно сделать для них правила из-за специфики данных отзывов, где могут быть совершенно разные по сттроению тексты

2. Составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec)
Мне этот метод нравится больше всего. Он не требует большой ручной работы. Проблемы: возможно плохое определение брендов, что не подойдет для второго задания. Попробуем реализоватть его, для создания словаря будем использовать извлечение ключевых слов и вектора Glove на твиттере

3. Можно попробывать кластеризацию на дополнителльных данных (которые меттадатта), однако на отзывах она может работать плохо опять же из-за специфики данных. Способ не очень надежный, однако он может помочь с бонусным заданием

4. Мысли насчет бонусного задания: это либо кластеризация, либо можно попробовать собрать вручную из title или метаданнных названия продуктов (чаще всего они там описаны полностью) Например, если в отзыве слово "guitar", для него будем искать в title его окружение, типа Fender_Telecaster_guitar, и заменять guitar в оттзыве на это название. В теории звучит круто, на практике думаю будет не оччень - не ддля всех отзывов есть нормальное название.


# Данные и импорты

Я решил взять датасет музыкальных инструментов (иногда там попадались книги по вязанию крючком, пусть это будет погрешность)

In [1]:
!wget 'http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Musical_Instruments.json.gz'
!wget 'http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Musical_Instruments.json.gz'

--2021-12-04 20:30:00--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Musical_Instruments.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232750177 (222M) [application/octet-stream]
Saving to: ‘Musical_Instruments.json.gz’

Musical_Instruments 100%[===================>] 221.97M  87.6MB/s    in 2.5s    

2021-12-04 20:30:03 (87.6 MB/s) - ‘Musical_Instruments.json.gz’ saved [232750177/232750177]

--2021-12-04 20:30:03--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Musical_Instruments.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172497344 (165M) [application/octet-stream]
Saving to: ‘meta_Musical_Instruments.json.gz’

meta_Musical_Instru 100%[==

In [2]:
!pip install summa

     |████████████████████████████████| 54 kB 2.6 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=225326873301f8eada77f12af93c0252fc401fd9819d5816bc213838b1b355e4
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import gzip
import numpy as np
from collections import Counter
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim.downloader
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
def parse(path):
  false = False
  true = True
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


In [5]:
pd.reset_option('^display.', silent=True)

In [6]:
df = getDF('Musical_Instruments.json.gz')
df_meta = getDF('meta_Musical_Instruments.json.gz')

In [7]:
df3 = df_meta.fillna(np.nan)
df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
df_meta_clean = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows

In [8]:
df

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,90,False,"08 9, 2004",AXHY24HWOF184,0470536454,{'Format:': ' Paperback'},Bendy,Crocheting for Dummies by Karen Manthey & Susa...,Terrific Book for Learning the Art of Crochet,1092009600,NaN
1,4.0,2,True,"04 6, 2017",A29OWR79AM796H,0470536454,{'Format:': ' Hardcover'},Amazon Customer,Very helpful...,Four Stars,1491436800,NaN
2,5.0,NaN,True,"03 14, 2017",AUPWU27A7X5F6,0470536454,{'Format:': ' Paperback'},Amazon Customer,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,Five Stars,1489449600,NaN
3,4.0,NaN,True,"02 14, 2017",A1N69A47D4JO6K,0470536454,{'Format:': ' Paperback'},Christopher Burnett,My girlfriend use quite often,Four Stars,1487030400,NaN
4,5.0,NaN,True,"01 29, 2017",AHTIQUMVCGBFJ,0470536454,{'Format:': ' Paperback'},Amazon Customer,Arrived as described. Very happy.,Very happy.,1485648000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1512525,5.0,NaN,True,"06 11, 2018",A311NUWMD55WPO,B01HJETSF2,{'Color:': ' Black (2 Pack)'},Jeffrey,Hard to believe anyone will play their prized ...,Easy to install and does the job,1528675200,NaN
1512526,4.0,NaN,True,"06 7, 2018",AXJFL8T55OXJG,B01HJETSF2,{'Color:': ' Black (2 Pack)'},ryan,"Straplocks installed nicely, good fitment on m...",Worth.,1528329600,NaN
1512527,5.0,NaN,True,"06 3, 2018",A1RHBI07JTQBER,B01HJETSF2,{'Color:': ' Black (2 Pack)'},Cyclonus,Great Product,Great Product,1527984000,NaN
1512528,5.0,NaN,True,"05 29, 2018",A1V0G9QMGY1RGK,B01HJETSF2,{'Color:': ' Black (2 Pack)'},Zach Smith,I honestly expected these to be cheap pieces o...,A pleasant surprise,1527552000,NaN


In [9]:
df_meta_clean

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Musical Instruments, Instrument Accessories, ...",,[Vivaldi's famous set of four violin concertos...,,Hal Leonard Vivaldi Four Seasons for Piano (Or...,[],,Hal Leonard,"[., ., .]","[>#330,653 in Musical Instruments (See Top 100...",[],Musical Instruments,,"May 10, 2011",$62.93,0041291905,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[The Turn of the Screw (op. 54) vocal score, p...",,The Turn of the Screw (vocal score),"[0486266842, 0793507669, 0393008789, 142341280...",,Boosey &amp; Hawkes,[],"[>#86,354 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"May 23, 2007",$107.79,0060015500,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,[],,[],,Suite for Organ (including the Trumpet Volunta...,[],,,[],"[>#482,025 in Musical Instruments (See Top 100...",[],Musical Instruments,,"February 8, 2013",,0193757710,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120305,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 10 pcs 3pdt ...,,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#23,334 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$7.99,B01HJDOF2Y,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120306,"[Musical Instruments, Amplifiers & Effects, Gu...",,[This packages includes:<br> <br> 5 pcs 3pdt T...,,"5 pcs 3pdt Stomp Footswitch incl. PCB, metal w...","[B015334J9G, B01L6GU858, B003HLW14W, B012CF181...",,3pdt,[5 pcs 9 Pole Footswitch for True Bypass Switc...,"[>#40,026 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"June 25, 2016",$0.91,B01HJDJ1PA,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120307,"[Musical Instruments, Drums & Percussion, Hand...",,[These high quality clear crystal singing bowl...,,Optical Clear Quartz Crystal Singing Bowl Note...,[],,Crystal Energy Bowls,[High quality clear crystal singing bowl is wi...,"[>#481,984 in Musical Instruments (See Top 100...",[],Musical Instruments,,"June 25, 2016",,B01HJEFFTK,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
120308,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,2x Deluxe Small 2&quot; Wood Frog Guiro Rasp -...,[],,RA,"[Wood percussion, Small 2 inches, Packs of 2 P...","[>#76,917 in Musical Instruments (See Top 100 ...","[B00NP80XMO, B00NP8GYVS, 0000098906, B0149IWDS...",Musical Instruments,"class=""a-bordered a-horizontal-strip

In [10]:
def preprocessing(text):
    text = text.lower()
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(BAD_SYMBOLS_RE, '', text)
    clean_text = ' '.join([word for word in text.split() if word and word not in STOPWORDS])
    tokens = list(nltk.word_tokenize(clean_text))
    lemmas = ' '.join([LEMMATIZER.lemmatize(word) for word in tokens])
    return lemmas

Склеим отзывы и метаданные, а также возьмем по одному отзыву на товар (очень долго работает обработка)

In [11]:
data = pd.DataFrame()
data = pd.merge(df[['asin', 'reviewText', 'summary']],
                df_meta_clean[['asin', 'category', 'title', 'description']],
                on='asin')
data = data.groupby('asin').first().reset_index()
data = data[~data['reviewText'].isna()]

In [12]:
data['text_clean'] = data['reviewText'].progress_apply(preprocessing)
data['title_clean'] = data['title'].progress_apply(preprocessing)
data['category_clean'] = data['category'].progress_apply(lambda x: preprocessing(' '.join(x)))

100%|██████████| 112118/112118 [00:20<00:00, 5352.60it/s]


In [13]:
data

,asin,reviewText,summary,category,title,description,text_clean,title_clean,category_clean
0,0000098906,Love this instuments! I love wooden toys so mu...,Love!,"[Musical Instruments, Drums & Percussion, Hand...","Wooden Percussion 3 Piece Set Frog, Cricket an...",[Frog - Rubbing its spine with the wooden stic...,love instuments love wooden toy much kid adult...,wooden percussion 3 piece set frog cricket owl...,musical instrument drum percussion hand percus...
1,0000989983,"Beautiful pieces, amazing quality, great sound!!!",Five Stars,"[Musical Instruments, Drums & Percussion, Hand...",Wooden Percussion 2 Piece Set of 3 Inch Cricke...,[Cricket Rubbing the spine with the wooden sti...,beautiful piece amazing quality great sound,wooden percussion 2 piece set 3 inch cricket i...,musical instrument drum percussion hand percus...
2,0041291905,This is and excellent edition and perfectly tr...,Vivalldi's Four Seasons,"[Musical Instruments, Instrument Accessories, ...",Hal Leonard Vivaldi Four Seasons for Piano (Or...,[Vivaldi's famous set of four violin concertos...,excellent edition perfectly true orchestral ve...,hal leonard vivaldi four season piano original...,musical instrument instrument accessory genera...
3,0060015500,It got here ahead of schedule and was in perfe...,Perfect scores & speedy delivery!,[],The Turn of the Screw (vocal score),"[The Turn of the Screw (op. 54) vocal score, p...",got ahead schedule perfect condition thankful ...,turn screw vocal score,
4,0193757710,Great music; fabulous arrangement! It is too b...,Great music!,[],Suite for Organ (including the Trumpet Volunta...,[],great music fabulous arrangement bad gone prin...,suite organ including trumpet voluntary sheet ...,
...,...,...,...,...,...,...,...,...,...
112127,B01HJDJ1PA,Great switch and board for a small quantity price,Five Stars,"[Musical Instruments, Amplifiers & Effects, Gu...","5 pcs 3pdt Stomp Footswitch incl. PCB, metal w...",[This packages includes:<br> <br> 5 pcs 3pdt T...,great switch board small quantity price,5 pc 3pdt stomp footswitch incl pcb metal wash...,musical instrument amplifier effect guitar bas...
112128,B01HJDOF2Y,"Good price for what you get, but out of the 10...",Decent value,"[Musical Instruments, Amplifiers & Effects, Gu...",10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[This packages includes:<br> <br> 10 pcs 3pdt ...,good price get 10 least 1 switch doesnt work h...,10 pc 3pdt stomp footswitch incl pcb incl meta...,musical instrument amplifier effect guitar bas...
112129,B01HJEFFTK,"beautiful, sounds lovely, but it has chips and...","chips and flaws, still sounds wonderful","[Musical Instruments, Drums & Percussion, Hand...",Optical Clear Quartz Crystal Singing Bowl Note...,[These high quality clear crystal singing bowl...,beautiful sound lovely chip flaw actually litt...,optical clear quartz crystal singing bowl note...,musical instrument drum percussion hand percus...
112130,B01HJEHEH6,Got these for a good price. Even though they ...,Got these for a good price. Even though they a...,"[Musical Instruments, Drums & Percussion, Hand...",2x Deluxe Small 2&quot; Wood Frog Guiro Rasp -...,[Frog - Rubbing its spine with the wooden stic...,got good price even though small make nice sou...,2x deluxe small 2quot wood frog guiro rasp mus...,musical instrument drum percussion hand percus...


# Метод

In [14]:
data = data.reset_index(drop=True)

Хочу взять пересечение ключевых слов отзывов и слов в категории (там хорошо описаны категории, например drums будут в категории, содержащей drums

In [15]:
def add_to_dict(el, d):
    if el in d:
        d[el] += 1
    else:
        d[el] = 1

d = {}
for i in tqdm(range(len(data))):
    kw = keywords.keywords(data['text_clean'][i], additional_stopwords=STOPWORDS, scores=True)
    for k in kw:
        add_to_dict(k[0], d)
        if k[0] in data['category_clean'][i]:
            d[k[0]] += 1

100%|██████████| 112118/112118 [12:53<00:00, 144.93it/s]


In [16]:
c = Counter(d).most_common(300)

Составим мини словарик

In [17]:
vocab = [
         'guitar', 'snare', 'banjo', 'recorder', 'flute', 'piano', 
         'ukulele', 'cymbal', 'microphone', 'violin', 'speaker', 'pedal',
         'instrument', 'headphone', 'tube', 'fender', 'tuner', 'cable', 
         'drum', 'bass', 'case', 'amp'
         ]

Возьмем glove-twitter-100, для отзывов мне кажется подойдет, и она небольшая по размеру

In [22]:
Glove = gensim.downloader.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


Посмотрим ближайшие

In [23]:
Glove.most_similar('guitar')[:5]

[('piano', 0.8171257972717285),
 ('drums', 0.8020161390304565),
 ('drum', 0.7792136669158936),
 ('bass', 0.7664357423782349),
 ('ukulele', 0.7589637637138367)]

In [24]:
Glove.most_similar('fender')[:5]

[('squier', 0.7491130828857422),
 ('telecaster', 0.733423113822937),
 ('strat', 0.7168670296669006),
 ('stratocaster', 0.6811788082122803),
 ('gibson', 0.6569886207580566)]

In [25]:
Glove.most_similar('bass')[:5]

[('drum', 0.8097249269485474),
 ('guitar', 0.7664357423782349),
 ('drums', 0.7193520069122314),
 ('jazz', 0.6832695007324219),
 ('sound', 0.6787845492362976)]

Сделаем дополнение к нашему списку

In [26]:
emb_vocab = []
for word in vocab:
  if word not in emb_vocab:
    emb_vocab.append(word)
    index = 0
    if word in Glove:
      while index != -1:
        nearest = Glove.most_similar(word)[index][0]
        if word == preprocessing(nearest):
          index += 1
        else:
          emb_vocab.append(nearest)
          index = -1

In [27]:
emb_vocab = set(emb_vocab)

In [28]:
print(emb_vocab)

{'pump', 'speaker', 'headphone', 'violin', 'tuner', 'chromatic', 'trumpet', 'flute', 'hdmi', 'cable', 'banjo', 'electric', 'note', 'pedal', 'guitar', 'flash', 'drum', 'cassette', 'fender', 'ukulele', 'instrument', 'synth', 'recorder', 'piano', 'cymbal', 'bass', 'mic', 'earphone', 'snare', 'amp', 'case', 'squier', 'microphone', 'tube', 'hi-hat'}


# N-граммы

## Биграммы

In [29]:
bigrams = []
for i, text in enumerate(tqdm(data['text_clean'])):
  text = text.split()
  for v in emb_vocab:
    idx = [i for i, word in enumerate(text) if word == v]
    if idx:
      for i in idx:
        if i != 0:
          bigrams.append(text[(i - 1):(i + 1)])
        if i != len(text) - 1:
          bigrams.append(text[i:(i + 2)])

100%|██████████| 112118/112118 [00:11<00:00, 9362.14it/s] 


In [30]:
bigrams

[['handwritten', 'note'],
 ['note', 'line'],
 ['taking', 'note'],
 ['note', 'music'],
 ['cd', 'tuner'],
 ['tuner', 'mostly'],
 ['well', 'piano'],
 ['piano', 'book'],
 ['classical', 'guitar'],
 ['guitar', '2'],
 ['pay', 'flute'],
 ['flute', 'really'],
 ['tune', 'instrument'],
 ['instrument', 'general'],
 ['selection', 'guitar'],
 ['guitar', 'solo'],
 ['good', 'note'],
 ['note', 'outlined'],
 ['clavinova', 'piano'],
 ['piano', 'musician'],
 ['module', 'piano'],
 ['five', 'note'],
 ['note', 'shire'],
 ['played', 'recorder'],
 ['recorder', 'much'],
 ['organization', 'piano'],
 ['piano', 'keyboard'],
 ['optional', 'piano'],
 ['piano', 'accompaniment'],
 ['accompanying', 'note'],
 ['note', 'difficult'],
 ['guide', 'note'],
 ['note', '6string'],
 ['bass', 'guitar'],
 ['guitar', 'besides'],
 ['musical', 'instrument'],
 ['instrument', 'could'],
 ['6string', 'bass'],
 ['bass', 'guitar'],
 ['move', 'drum'],
 ['drum', 'machine'],
 ['mean', 'cassette'],
 ['cassette', '4track'],
 ['source', 'instrum

In [31]:
bigram_metrics = nltk.collocations.BigramAssocMeasures()
bigrams_finder = nltk.collocations.BigramCollocationFinder.from_documents(bigrams)
bigrams_finder.apply_freq_filter(10)

In [32]:
dice = bigrams_finder.score_ngrams(bigram_metrics.dice)
pmi = bigrams_finder.score_ngrams(bigram_metrics.pmi)
chi_sq = bigrams_finder.score_ngrams(bigram_metrics.chi_sq)

print('Dice', dice[:30])
print('PMI', pmi[:30])
print('Chi_sq', chi_sq[:30])

Dice [(('flash', 'drive'), 0.16), (('chromatic', 'harmonica'), 0.1394422310756972), (('cassette', 'deck'), 0.10679611650485436), (('cassette', 'tape'), 0.10084033613445378), (('acoustic', 'electric'), 0.0903970712475359), (('headphone', 'jack'), 0.07922705314009662), (('snare', 'drum'), 0.07706635622817229), (('tube', 'amp'), 0.07361856920145506), (('squier', 'affinity'), 0.06866952789699571), (('mic', 'stand'), 0.06471783648585275), (('digital', 'piano'), 0.05987021142976764), (('drum', 'set'), 0.05481671504759333), (('audio', 'cassette'), 0.05405405405405406), (('bass', 'drum'), 0.053866871873797614), (('squier', 'strat'), 0.052691867124856816), (('pedal', 'board'), 0.04959687270950403), (('grand', 'piano'), 0.04755728491137051), (('trumpet', 'player'), 0.04696531791907514), (('light', 'flash'), 0.04582651391162029), (('analog', 'synth'), 0.04328018223234624), (('pocket', 'trumpet'), 0.04233870967741935), (('ride', 'cymbal'), 0.04128938790293372), (('usb', 'flash'), 0.040968342644320

Лучше всего показал себя Dice

In [33]:
coll_dict = {}
for v in emb_vocab:
  coll_dict[v] = []

for bigram in dice:
  words = set(list(bigram[0])) & emb_vocab
  for word in words:
    coll_dict[word].append(' '.join(bigram[0]))

for v in emb_vocab:
  coll_dict[v] = list(set(coll_dict[v]))

In [34]:
for word in ['guitar', 'piano', 'drum', 'violin', 'trumpet', 'fender']:
    print(word)
    for i in coll_dict[word][:10]:
      print('\t'+i)

guitar
	guitar isnt
	previous guitar
	compared guitar
	guitar another
	since guitar
	jack guitar
	ukulele guitar
	esp guitar
	guitar shop
	favorite guitar
piano
	piano tuner
	piano lesson
	piano teacher
	upright piano
	piano im
	key piano
	practice piano
	grand piano
	piano synth
	piano violin
drum
	drum snare
	drum right
	roland drum
	addition drum
	drum circle
	using drum
	drum mount
	drum many
	drum sounded
	drum son
violin
	violin made
	violin 4
	protect violin
	electric violin
	violin bow
	violin beautiful
	like violin
	violin well
	fine violin
	get violin
trumpet
	piccolo trumpet
	good trumpet
	professional trumpet
	trumpet valve
	trumpet mute
	trumpet flugelhorn
	played trumpet
	trumpet player
	great trumpet
	pocket trumpet
fender
	tuner fender
	fender us
	fender squire
	well fender
	fender logo
	genuine fender
	standard fender
	fender bass
	good fender
	clean fender


## Триграммы

In [35]:
trigrams = []
for i, text in enumerate(tqdm(data['text_clean'])):
  text = text.split()
  for v in emb_vocab:
    idx = [i for i, word in enumerate(text) if word == v]
    if idx:
      for i in idx:
        if i != 0:
          trigrams.append(text[(i - 1):(i + 2)])

100%|██████████| 112118/112118 [00:12<00:00, 8794.40it/s] 


In [36]:
trigrams

[['handwritten', 'note', 'line'],
 ['taking', 'note', 'music'],
 ['cd', 'tuner', 'mostly'],
 ['well', 'piano', 'book'],
 ['classical', 'guitar', '2'],
 ['pay', 'flute', 'really'],
 ['tune', 'instrument', 'general'],
 ['selection', 'guitar', 'solo'],
 ['good', 'note', 'outlined'],
 ['clavinova', 'piano', 'musician'],
 ['module', 'piano'],
 ['five', 'note', 'shire'],
 ['played', 'recorder', 'much'],
 ['organization', 'piano', 'keyboard'],
 ['optional', 'piano', 'accompaniment'],
 ['accompanying', 'note', 'difficult'],
 ['guide', 'note', '6string'],
 ['bass', 'guitar', 'besides'],
 ['musical', 'instrument', 'could'],
 ['6string', 'bass', 'guitar'],
 ['move', 'drum', 'machine'],
 ['mean', 'cassette', '4track'],
 ['source', 'instrument', 'big'],
 ['make', 'instrument', 'sound'],
 ['straight', 'instrument', 'recorder'],
 ['instrument', 'recorder', 'room'],
 ['usually', 'mic', 'small'],
 ['7', 'case', 'work'],
 ['stringed', 'instrument', '30'],
 ['play', 'guitar', 'thanks'],
 ['easy', 'piano'

In [37]:
trigrams_metrics = nltk.collocations.TrigramAssocMeasures()
trigrams_finder = nltk.collocations.TrigramCollocationFinder.from_documents(trigrams)
trigrams_finder.apply_freq_filter(5)

In [38]:
jaccard = trigrams_finder.score_ngrams(trigrams_metrics.jaccard)
pmi = trigrams_finder.score_ngrams(trigrams_metrics.pmi)
chi_sq = trigrams_finder.score_ngrams(trigrams_metrics.chi_sq)

print('Jaccard', jaccard[:30])
print('PMI', pmi[:30])
print('Chi_sq', chi_sq[:30])

Jaccard [(('usb', 'flash', 'drive'), 0.019522776572668113), (('wireless', 'microphone', 'system'), 0.0052310374891020054), (('professional', 'trumpet', 'player'), 0.0046559751681324365), (('ibanez', 'tube', 'screamer'), 0.004482869036183158), (('taking', 'piano', 'lesson'), 0.0031708283789139914), (('local', 'violin', 'shop'), 0.002875816993464052), (('wireless', 'mic', 'system'), 0.002722177742193755), (('daughter', 'violin', 'teacher'), 0.0027196083763937995), (('cloth', 'amp', 'feature'), 0.002436548223350254), (('acoustic', 'electric', 'guitar'), 0.0022038959192378785), (('inch', 'headphone', 'jack'), 0.00218435998252512), (('im', 'trumpet', 'player'), 0.0021331058020477816), (('groove', 'tube', '12ax7'), 0.0019933554817275745), (('electronic', 'drum', 'kit'), 0.0019541427826993227), (('started', 'piano', 'lesson'), 0.001946282600233554), (('delight', 'violin', 'wise'), 0.0017321016166281756), (('locking', 'tuner', 'make'), 0.001719526175009553), (('standard', 'mic', 'stand'), 0.00

Мне нравится Jacard

In [39]:
coll_dict = {}
for v in emb_vocab:
  coll_dict[v] = []

for trigram in jaccard:
  words = set(list(trigram[0])) & emb_vocab
  for word in words:
    coll_dict[word].append(' '.join(trigram[0]))

for v in emb_vocab:
  coll_dict[v] = list(set(coll_dict[v]))

In [40]:
for word in ['guitar', 'piano', 'drum', 'violin', 'trumpet', 'fender']:
  print(word)
  for i in coll_dict[word][:10]:
    print('\t'+i)

guitar
	love guitar much
	amazing guitar ive
	way guitar sound
	make guitar pick
	fan guitar player
	purchased guitar center
	electric guitar im
	guitar electric acoustic
	fit guitar case
	electric guitar fit
piano
	grand piano sound
	taking piano lesson
	digital piano sound
	real piano sound
	piano electric piano
	started piano lesson
	good piano sound
drum
	use drum circle
	bass drum mic
	bass drum fit
	fit bass drum
	great drum set
	kick drum microphone
	bass drum bag
	great snare drum
	drum bass guitar
	bought drum set
violin
	love violin case
	delight violin wise
	daughter violin teacher
	electric violin shop
	violin case bow
	local violin shop
	designer violin playing
	taking violin lesson
trumpet
	im trumpet player
	professional trumpet player
fender
	fit fender strat
	love fender guitar
	fit fender jazz
	fender tube amp
	fender amp sound
	genuine fender part
	standard fender strat


Что можно сказать:
1. Биграммы могут показывать отношение к товару good violin, finne violin, либо же сопутствующие товары drum mount
2. Триграммы реже показывают отношение к ттовару, чаще - сопуствующие вещи bass drum mic, fit guitar case
3. Не получилось выделять бренды товаров, только если использовать сам бренд как слово: так получим fender amp sound (усилок) и fender squire (одна из моделей гитар). 

Вообще в музыкальной тусовке принято называть гитары по модели, например, Stratocaster или Telecaster, но для этих слов не вышло ничего показать.